# ETL y EDA archivo steam_games

In [2]:
# importar módulos
# import io
# import gzip
# import ast
import pandas as pd
# import json
# from google.colab import files
import fastparquet
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_colwidth', None)

### **Carga del datasets**

In [3]:
# cargar el dataset
df_steam_games = pd.read_parquet("archivos_originales//df_steam_games.parquet")

In [ ]:
df_steam_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0


In [ ]:
# función para obtener datos varios de los dataframe(info, describe, null, duplicated)
def print_df_info(df):

    print("** Info - Información general **")
    print(df.info())

    print("=======================================================")

    print("** Shape **")
    print(f"Filas: {df.shape[0]} - Columnas: {df.shape[1]}")

    print("=======================================================")

    print("** Describe - Estadísticas resumidas **")
    print(df.describe().T)

    print("=======================================================")

    print("** NaN - Nulos **")
    print(df.isna().sum())

    print("=======================================================")
    print("** Duplicated - Duplicados **")
    print(df.duplicated().sum())

In [ ]:
print_df_info(df_steam_games)

** Info - Información general **
<class 'pandas.core.frame.DataFrame'>
Int64Index: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24073 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2607 non-null   float64
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  float64
 

In [ ]:
# columnas
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items', 'items_count'],
      dtype='object')

In [4]:
# eliminar columnas
df_steam_games.drop(
    [
        "publisher", "app_name", "title", "url", "tags", "reviews_url", "discount_price", "specs", "early_access", "metascore",
        "user_id", "steam_id", "items", "items_count"
    ], axis=1, inplace=True)

In [5]:
df_steam_games.shape

(120445, 5)

In [6]:
# eliminar duplicados
df_steam_games.drop_duplicates(inplace=True)

In [7]:
# eliminar primera fila, esta se crea despues de eliminar duplicados
df_steam_games = df_steam_games.drop([0])

In [8]:
# verifico la eliminación de la fila
df_steam_games.head(2)

,genres,release_date,price,id,developer
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04,4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018-01-04,Free To Play,643980.0,Secret Level SRL


In [9]:
# reset del índice
df_steam_games.reset_index(drop=True, inplace=True)

In [10]:
# extraer el año
df_steam_games['year'] = df_steam_games['release_date'].str.extract(r'(\d{4})')

In [11]:
df_steam_games.head()

,genres,release_date,price,id,developer,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04,4.99,761140.0,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018-01-04,Free To Play,643980.0,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",2017-07-24,Free to Play,670290.0,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",2017-12-07,0.99,767400.0,彼岸领域,2017
4,None,None,2.99,773570.0,None,NaN


In [22]:
# eliminar nulos columna price - los valores nan, none no se consideran como contenido Free
df_steam_games[df_steam_games["price"].isnull()]
df_steam_games.dropna(subset=["price"], inplace=True)

,genres,release_date,price,id,developer,year


In [13]:
# ver cantidad de Free, Free To Play y Free to Play
print(df_steam_games[df_steam_games["price"] == "Free"].count().sum())
print(df_steam_games[df_steam_games["price"] == "Free To Play"].count().sum())
print(df_steam_games[df_steam_games["price"] == "Free to Play"].count().sum())

4598
2403
2930


In [14]:
# eliminar espacios en blanco y extraer la palabra Free para estandarizar el contenido
df_steam_games['price'] = df_steam_games['price'].str.strip()
df_steam_games['price'] = df_steam_games['price'].str.replace(' To Play', '')
df_steam_games['price'] = df_steam_games['price'].str.replace(' to Play', '')

In [15]:
# Verificar qué filas contienen letras
rows_with_letters = df_steam_games['price'].apply(lambda x: any(c.isalpha() for c in str(x)))

# Contar cuántas filas contienen letras
count_letters = sum(rows_with_letters)

print(f"El número de filas que contienen letras es: {count_letters}")

El número de filas que contienen letras es: 1910


In [25]:
# Filtrar las filas que contienen letras en la columna "price"
rows_with_letters = df_steam_games[df_steam_games['price'].str.contains('[a-zA-Z]', regex=True)]

# Imprimir las filas que contienen letras en la columna "price"
rows_with_letters[rows_with_letters.price != "Free"]
# print(rows_with_letters.to_string())

,genres,release_date,price,id,developer,year
920,['Indie'],2012-04-06,Free Demo,213530.0,TAO,2012
921,"['Indie', 'Casual']",2012-04-06,Free Demo,213390.0,BitSits Games,2012
969,"['Free to Play', 'Massively Multiplayer', 'RPG']",2012-06-06,Play for Free!,212500.0,"Standing Stone Games, LLC",2012
981,"['Action', 'Adventure', 'Free to Play', 'Massi...",2012-06-25,Play for Free!,206480.0,"Standing Stone Games, LLC",2012
2405,['Utilities'],2014-09-19,Install Now,268850.0,EVGA,2014
2871,"['Indie', 'Strategy']",2014-11-20,Play WARMACHINE: Tactics Demo,253510.0,WhiteMoon Dreams,2014
3314,"['Action', 'Free to Play', 'RPG']",2008-01-18,Free Mod,349480.0,Ascension Entertainment LLC,2008
3832,"['Adventure', 'Casual', 'Indie', 'RPG', 'Simul...",2015-06-10,Install Theme,370880.0,Stolen Couch Games,2015
3918,['Indie'],2015-07-02,Third-party,362970.0,Polar Bunny Ltd,2015
4026,"['Casual', 'Indie']",2015-07-20,Play Now,345040.0,"CorypheeSoft,DigitalEZ",2015


In [26]:
# reemplazar Free por cero(0) - los valores nan, none no se consideraron como contenido Free
# algunos valores se consideran Free tales como, 0 Demo, Play for 0!, etc
df_steam_games['price'] = df_steam_games['price'].str.replace('Free', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('0 Demo', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('Play for 0!', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('Play WARMACHINE: Tactics Demo', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('0 Mod', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('0 HITMAN™ Holiday Pack', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('Play the Demo', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('0 to Try', '0')
df_steam_games['price'] = df_steam_games['price'].str.replace('0 to Use', '0')

In [41]:
# se detectaron otras cadenas alphanuméricas de las cuales algunas se consideran Free y
# las otras se eliminan
df_steam_games = df_steam_games.drop([2405, 3314, 3832, 3918, 31837, 4026, 26216, 24999, 25000, 29365])

In [40]:
df_steam_games[df_steam_games['price'] == "Install Now"]
df_steam_games.loc[29365, :]

genres                None
release_date    2014-03-19
price              0 Movie
id                245550.0
developer             None
year                  2014
Name: 29365, dtype: object

In [42]:
# cambio del tipo de dato de la columna
df_steam_games['price'] = df_steam_games['price'].astype(float)

In [43]:
# eliminar la columna release_date, solo se necesita el año
df_steam_games.drop(["release_date"], axis=1, inplace=True)

In [44]:
# verifico
df_steam_games.head()

,genres,price,id,developer,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",4.99,761140.0,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",0.00,643980.0,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",0.00,670290.0,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",0.99,767400.0,彼岸领域,2017
4,None,2.99,773570.0,None,NaN


In [45]:
print(df_steam_games.isnull().sum())
df_steam_games[df_steam_games['genres'].isnull()]

genres       3133
price           0
id              2
developer    3154
year         1940
dtype: int64


,genres,price,id,developer,year
4,None,2.99,773570.0,None,None
11,None,0.00,724910.0,None,None
19,None,4.99,772590.0,None,None
20,None,2.99,640250.0,None,None
22,None,0.99,711440.0,None,None
...,...,...,...,...,...
32117,None,3.99,775640.0,None,2018
32118,None,3.99,777930.0,None,2018
32119,None,4.99,775370.0,None,2016
32120,None,0.00,777950.0,None,2018


In [46]:
df_steam_games.dropna(subset=["genres"], inplace=True)
df_steam_games.isnull().sum()

genres         0
price          0
id             1
developer    155
year           7
dtype: int64

In [47]:
df_steam_games.dropna(subset=["developer"], inplace=True)
df_steam_games.isnull().sum()

genres       0
price        0
id           1
developer    0
year         7
dtype: int64

In [48]:
df_steam_games.dropna(subset=["year"], inplace=True)
df_steam_games.isnull().sum()

genres       0
price        0
id           1
developer    0
year         0
dtype: int64

In [49]:
df_steam_games.dropna(subset=["id"], inplace=True)
df_steam_games.isnull().sum()

genres       0
price        0
id           0
developer    0
year         0
dtype: int64

In [50]:
df_steam_games.shape

(27451, 5)

In [51]:
df_steam_games.head()

,genres,price,id,developer,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",4.99,761140.0,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",0.00,643980.0,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",0.00,670290.0,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",0.99,767400.0,彼岸领域,2017
5,"['Action', 'Adventure', 'Simulation']",3.99,772540.0,Trickjump Games Ltd,2018


In [52]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27451 entries, 0 to 32132
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     27451 non-null  object 
 1   price      27451 non-null  float64
 2   id         27451 non-null  float64
 3   developer  27451 non-null  object 
 4   year       27451 non-null  object 
dtypes: float64(2), object(3)
memory usage: 1.3+ MB


In [53]:
# cambiar el tipo de dato de la columna id y year
df_steam_games['id'] = df_steam_games['id'].astype(int)
df_steam_games['year'] = df_steam_games['year'].astype(int)

In [54]:
# cambiar el nombre columna
df_steam_games = df_steam_games.rename(columns={'id':'item_id'})

In [55]:
# reset del índice
df_steam_games.reset_index(drop=True, inplace=True)

In [56]:
df_steam_games.head(2)

,genres,price,item_id,developer,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",4.99,761140,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",0.00,643980,Secret Level SRL,2018


In [57]:
# exportar a parquet ya limpio el archivo
df_steam_games_2 = df_steam_games.to_parquet("datasets/df_steam_games_2.parquet")